In [1]:
!pip install timm==0.9.12


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
pwd

In [3]:
import os
import gc
import typing as t
import numpy as np
import pandas as pd
from tqdm import tqdm

from fastai.vision.all import *

import timm
import torch
from torch import nn, optim

import torchvision
from torchvision import datasets, transforms

from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split

from sklearn.model_selection import StratifiedKFold

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# del model, learn
# gc.collect()

In [6]:
def clean_cuda_mem():
    gc.collect()
    torch.cuda.empty_cache()

In [7]:
clean_cuda_mem()

In [8]:
def get_batch_size(
    model: nn.Module,
    device: torch.device,
    input_shape: t.Tuple[int, int, int],
    output_shape: t.Tuple[int],
    dataset_size: int,
    max_batch_size: int = None,
    num_iterations: int = 5,
) -> int:
    model.to(device)
    model.train(True)
    optimizer = torch.optim.Adam(model.parameters())

    batch_size = 2
    while True:
        if max_batch_size is not None and batch_size >= max_batch_size:
            batch_size = max_batch_size
            break
        if batch_size >= dataset_size:
            batch_size = batch_size // 2
            break
        try:
            for _ in range(num_iterations):
                # dummy inputs and targets
                inputs = torch.rand(*(batch_size, *input_shape), device=device)
                targets = torch.rand(*(batch_size, *output_shape), device=device)
                outputs = model(inputs)
                loss = F.mse_loss(targets, outputs)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            batch_size *= 2
        except RuntimeError:
            batch_size //= 2
            break
    del model, optimizer
    torch.cuda.empty_cache()
    return batch_size

In [9]:
set_seed(13, reproducible=True)

In [10]:
path = "../data"

In [11]:
train_df = pd.read_csv(f"{path}/train/train/train.csv")
test_df = pd.read_csv(f"{path}/test/test/test.csv")

train_df.shape, test_df.shape

((8079, 3), (3462, 2))

In [12]:
target = 'label'

train_df[target].value_counts(normalize=True)

label
0    0.942443
1    0.057557
Name: proportion, dtype: float64

In [13]:
train_df['filename'] = train_df['filename'].apply(lambda x: f"{path}/train/train/images/" + x)
test_df['filename'] = test_df['filename'].apply(lambda x: f"{path}/test/test/images/" + x)

train_df.head()

,image_id,filename,label
0,1,../data/train/train/images/1.jpg,0
1,2,../data/train/train/images/2.jpg,0
2,3,../data/train/train/images/3.jpg,0
3,4,../data/train/train/images/4.jpg,0
4,5,../data/train/train/images/5.jpg,0


In [14]:
# train_df = train_df.sample(500)
# test_df = test_df.sample(100)

In [15]:
img_size = (512, 512) # 384, 224

In [16]:
model_name = 'maxvit_large_tf_512' # swinv2_large_window12to24_192to384, convnextv2_base, convnextv2_large, vit_small_patch16_224, swin_large_patch4_window12_384

# dataset information
IMAGE_SHAPE = (3, *img_size)
NUM_CLASSES = 2
DATASET_SIZE = train_df.shape[0]

print(IMAGE_SHAPE)

(3, 512, 512)


In [17]:
# model = timm.create_model(
#     model_name, 
#     pretrained=True, 
#     num_classes=2) # dls.c

model.safetensors:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [18]:
# device = torch.device("cuda")

# batch_size = get_batch_size(
#     model=model,
#     device=device,
#     input_shape=IMAGE_SHAPE,
#     output_shape=(NUM_CLASSES,),
#     dataset_size=DATASET_SIZE,
# )
# batch_size

4

In [19]:
batch_size = 4

In [20]:
clean_cuda_mem()

In [78]:
# timm.list_models('*convnextv2*')

In [21]:
# dls = ImageDataLoaders.from_df(
#     train_df,
#     fn_col='filename',
#     label_col='label',
#     valid_pct=0.2, 
#     seed=13,
#     shuffle=True,
#     item_tfms=Resize(size=img_size),
#     bs=batch_size,
#     batch_tfms=setup_aug_tfms([
#         Brightness(), 
#         Contrast(), 
#         Hue(), 
#         Saturation(),
        
#         Flip(),
#         Zoom(),
# #         Warping(),
        
#     ])
# )

# test_dl = dls.test_dl(test_df)

# dls.show_batch()

In [21]:
def create_model(dl, model, metrics):
    learn = Learner(dl, model, metrics=metrics)
    
    return learn

def fit_model(learn, max_epochs, lr, cbs):
    learn.fit_one_cycle(
        max_epochs,
        lr,
        cbs=cbs
    )
    
    return learn

def get_preds(dl, learn, get_tta=False):
    preds, _ = learn.get_preds(dl=test_dl)
    preds = preds[:, 1].numpy()
    
    if get_tta:
        tta_preds, _ = learn.tta(dl=test_dl)
        tta_preds = tta_preds[:, 1].numpy()
        
        return preds, tta_preds
    
    return preds

In [22]:
def create_folds(train_df, N_FOLDS=5):
    train_df['fold'] = -1
    
    strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=13, shuffle=True)
    for i, (_, trn_idx) in enumerate(strat_kfold.split(train_df.index, train_df[target])):
        train_df.iloc[trn_idx, -1] = i

    train_df['fold'] = train_df['fold'].astype('int')

#     train_df.fold.value_counts().plot.bar()
    
    return train_df

In [23]:
def create_model(dl, model, metrics):
    learn = Learner(dl, model, metrics=metrics)
    
    return learn

def fit_model(learn, max_epochs, lr, cbs):
    learn.fit_one_cycle(
        max_epochs,
        lr,
        cbs=cbs
    )
    
    return learn

def get_preds(dls, learn, get_tta=False, valid=False):
    if valid:
        preds, _ = learn.get_preds()
    else:
        test_dl = dls.test_dl(test_df)
        preds, _ = learn.get_preds(dl=test_dl)
    preds = preds[:, 1].numpy()
    
    if get_tta:
        tta_preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
        tta_preds = tta_preds[:, 1].numpy()
        
        return preds, tta_preds
    
    return preds

def get_best_metrics(learn):
    acc, f1 = learn.recorder.metrics[0].value.item(), learn.recorder.metrics[1].value.item()
    
    return acc, f1

In [24]:
def get_data(fold):
    cur_train = train_df.copy()
    cur_train['is_valid'] = (cur_train['fold'] == fold)
    
    dls = ImageDataLoaders.from_df(
        cur_train,
        fn_col='filename',
        label_col='label',
        valid_col='is_valid',
        seed=13,
        shuffle=True,
        item_tfms=Resize(size=img_size),
        bs=batch_size,
        batch_tfms=setup_aug_tfms([
            Brightness(), 
            Contrast(), 
            Hue(), 
            Saturation(),

            Flip(),
            Zoom(),
    #         Warping(),

        ])
    )

    return dls

In [25]:
metrics=[accuracy, F1Score(average='macro')]

def build_callbacks(save_name):
    cbs=[
        EarlyStoppingCallback(monitor='f1_score', comp=np.greater, patience=3),
        SaveModelCallback(monitor='f1_score', comp=np.greater, fname=save_name, at_end=False), 
    #     ReduceLROnPlateau(monitor='f1_score', comp=np.greater, patience=2, factor=0.2, min_lr=1e-5),
    ]
    
    return cbs

In [26]:
max_epochs = 15
lr = 2e-5

In [27]:
# cbs = build_callbacks(save_name=model_name)

In [64]:
# model = timm.create_model(
#     model_name, 
#     pretrained=True, 
#     num_classes=2) # 

In [63]:
# learn = create_model(dls, model, metrics)
# # learn.lr_find(end_lr=3e-2)

In [62]:
# del model, learn

# gc.collect()

In [30]:
# clean_cuda_mem()

In [56]:
# learn = fit_model(learn, max_epochs, lr=lr, cbs=cbs)
# learn.recorder.plot_loss()

In [57]:
# preds, tta_preds = get_preds(dls, learn, get_tta=True)

In [58]:
# preds.shape, tta_preds.shape

In [59]:
# sub = pd.read_csv("../data/sample_submission.csv")
# sub['label'] = tta_preds # preds
# sub['label'] = sub['label'].apply(lambda x: 1 if x >= 0.5 else 0)
# sub['label'].value_counts(normalize=True)

In [60]:
# model_name

In [61]:
# sub.to_csv(f"{model_name}_384_2.csv", index=False)

In [28]:
N_FOLDS = 5

train_df = create_folds(train_df, N_FOLDS)

In [29]:
# dls = get_data(0)
# dls.show_batch()

In [ ]:
all_preds = [] # np.zeros(shape=(dataloader.test.shape[0], 1))
all_preds_tta = []
all_val_preds = []

score = 0

for i in tqdm(range(N_FOLDS)):

    print(f'Fold {i} results')

    dls = get_data(i)
    
    model = timm.create_model(
        model_name, 
        pretrained=True, 
        num_classes=dls.c)
    
    cbs = build_callbacks(save_name=f"{model_name}_fold{i}")

    learn = create_model(dls, model, metrics)# .to_fp16()
    
    learn = fit_model(learn, max_epochs, lr=lr, cbs=cbs)
    learn.recorder.plot_loss()
    
    fold_acc, fold_f1 = get_best_metrics(learn)
    print(f"Best Acc: {fold_acc} | \t Best F1 : {fold_f1}")
    
    score += fold_f1

    #learn = learn.to_fp32()

    #learn.export(f'model_fold_{i}.pkl')
    #learn.save(f'model_fold_{i}.pkl')
    
    preds, tta_preds = get_preds(dls, learn, get_tta=True, valid=False)
    val_preds = get_preds(dls, learn, get_tta=False, valid=True)

    all_preds.append(preds)
    all_preds_tta.append(tta_preds)
    all_val_preds.append(val_preds)

    del learn, dls, model
    torch.cuda.empty_cache()
    gc.collect()
    
    print('--'*30)

score /= N_FOLDS
print(f"Mean KFold Score for {model_name} : {score}")

  0%|          | 0/5 [00:00<?, ?it/s]

Fold 0 results


epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.126382,0.173825,0.943069,0.495979,18:16
1,0.069025,0.073068,0.973391,0.881526,18:17
2,0.017128,0.029116,0.989480,0.948088,18:20
3,0.005273,0.055256,0.987624,0.947678,18:21
4,0.014847,0.010150,0.997525,0.988355,18:20
5,0.001011,0.007107,0.998762,0.994237,18:21
6,0.001291,0.002624,0.998762,0.994237,18:21
7,0.000083,0.000585,1.000000,1.000000,18:20
8,0.000028,0.000921,0.999381,0.997162,18:20
9,0.000439,0.001581,0.999381,0.997162,18:20


Better model found at epoch 0 with f1_score value: 0.4959792793892302.
Better model found at epoch 1 with f1_score value: 0.8815263406458697.
Better model found at epoch 2 with f1_score value: 0.9480883661780739.
Better model found at epoch 4 with f1_score value: 0.9883552513060709.
Better model found at epoch 5 with f1_score value: 0.994237133401803.
Better model found at epoch 7 with f1_score value: 1.0.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 3 with f1_score value: 0.9856660073266572.
Better model found at epoch 4 with f1_score value: 0.9888162220146026.
Better model found at epoch 5 with f1_score value: 1.0.


In [ ]:
score

In [ ]:
score

In [ ]:
import joblib

joblib.dump(all_preds, f"{model_name}_all_preds.pkl")
joblib.dump(all_preds_tta, f"{model_name}_all_preds_tta.pkl")
joblib.dump(all_val_preds, f"{model_name}_all_val_preds.pkl")

In [ ]:
len(all_preds), len(all_preds_tta), len(all_val_preds)

In [ ]:
mean_preds = np.mean(np.stack(all_preds, axis=1), axis=1)
mean_preds_tta = np.mean(np.stack(all_preds_tta, axis=1), axis=1)

mean_preds.shape, mean_preds_tta.shape

In [33]:
# learner = Learner(get_data(0), model_name, metrics=[accuracy, F1Score(average='macro')])
# learner.fit_one_cycle(
#     20, 
#     2e-5, 
#     cbs=[EarlyStoppingCallback(monitor='f1_score', comp=np.greater, patience=3)]) # SaveModelCallback(), 

In [104]:
# preds, _ = learner.get_preds(dl=test_dl)
# preds = preds[:, 1].numpy()
# print(preds.shape)

# pd.Series(preds).plot()

In [ ]:
sub = pd.read_csv("../data/sample_submission.csv")
sub['label'] = mean_preds_tta
sub['label'] = sub['label'].apply(lambda x: 1 if x >= 0.5 else 0)
sub['label'].value_counts(normalize=True)

In [ ]:
sub.to_csv(f"{model_name}_5fold_512_2.csv", index=False)

In [ ]:
train_df.to_csv(f"{model_name}_train_df.csv", index=False)

In [ ]:
# 05:47PM

In [ ]:
1